In [25]:
# Hybrid CNN + Transformer for CIFAKE (Notebook Style)
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
from tqdm import tqdm  # <-- added for progress bar
from sklearn.metrics import classification_report


In [3]:
# ---- Setup ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 64
EPOCHS = 10
IMG_SIZE = 128
NUM_CLASSES = 2

In [4]:
# ---- Transforms ----
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor()
])

In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/cifake-real-and-ai-generated-synthetic-images


In [7]:
# ---- Dataset Load ----
full_train_dataset = datasets.ImageFolder('/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train', transform=transform)
class_map = full_train_dataset.class_to_idx
print("Class Mapping:", class_map)
real_idx = class_map['REAL']
fake_idx = class_map['FAKE']

Class Mapping: {'FAKE': 0, 'REAL': 1}


In [8]:
def remap_targets(dataset):
    for i in range(len(dataset.targets)):
        if dataset.targets[i] == fake_idx:
            dataset.targets[i] = 1
        elif dataset.targets[i] == real_idx:
            dataset.targets[i] = 0

remap_targets(full_train_dataset)

In [9]:
# ---- Split into Train and Val ----
train_len = int(0.8 * len(full_train_dataset))
val_len = len(full_train_dataset) - train_len
train_dataset, val_dataset = random_split(full_train_dataset, [train_len, val_len])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [10]:
# ---- Test Set ----
test_dataset = datasets.ImageFolder('/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test', transform=transform)
remap_targets(test_dataset)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [11]:
# ---- Hybrid Model ----
class HybridModel(nn.Module):
    def __init__(self):
        super(HybridModel, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.patch_embed = nn.Conv2d(256, 128, kernel_size=4, stride=4)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=128, nhead=4, dim_feedforward=256),
            num_layers=2
        )
        self.classifier = nn.Linear(128, NUM_CLASSES)

    def forward(self, x):
        x = self.cnn(x)
        x = self.patch_embed(x)         # [B, 128, 8, 8]
        x = x.flatten(2).permute(2, 0, 1)  # [64, B, 128]
        x = self.transformer(x)         # [64, B, 128]
        x = x.mean(dim=0)               # [B, 128]
        return self.classifier(x)


In [12]:
# ---- Train & Eval Functions ----
def train(model, loader, optimizer, criterion):
    model.train()
    running_loss, correct = 0, 0
    for x, y in tqdm(loader, desc="Training"):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        correct += (out.argmax(1) == y).sum().item()
    acc = correct / len(loader.dataset)
    return running_loss / len(loader), acc

In [13]:
def evaluate(model, loader, criterion):
    model.eval()
    loss, correct = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss += criterion(out, y).item()
            correct += (out.argmax(1) == y).sum().item()
    acc = correct / len(loader.dataset)
    return loss / len(loader), acc


In [14]:
# ---- Training ----
model = HybridModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_loader, criterion)
    print(f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(



Epoch 1/10


Training: 100%|██████████| 1250/1250 [12:58<00:00,  1.61it/s]


Train Acc: 0.8054 | Val Acc: 0.8879

Epoch 2/10


Training: 100%|██████████| 1250/1250 [04:50<00:00,  4.30it/s]


...

Train Acc: 0.9554 | Val Acc: 0.9462

Epoch 9/10


Training: 100%|██████████| 1250/1250 [04:02<00:00,  5.15it/s]


Train Acc: 0.9588 | Val Acc: 0.9504

Epoch 10/10


Training: 100%|██████████| 1250/1250 [04:04<00:00,  5.11it/s]


Train Acc: 0.9624 | Val Acc: 0.9539


In [15]:
# ---- Testing ----
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for x, y in tqdm(test_loader, desc="Testing"):
        x = x.to(device)
        out = model(x)
        preds = out.argmax(1).cpu()
        y_true.extend(y.numpy())
        y_pred.extend(preds.numpy())

print("\nTest Classification Report:")
print(classification_report(y_true, y_pred, target_names=["REAL", "FAKE"]))

Testing: 100%|██████████| 313/313 [02:42<00:00,  1.93it/s]


Test Classification Report:
              precision    recall  f1-score   support

        REAL       0.94      0.97      0.95     10000
        FAKE       0.96      0.94      0.95     10000

    accuracy                           0.95     20000
   macro avg       0.95      0.95      0.95     20000
weighted avg       0.95      0.95      0.95     20000



In [27]:
accuracy = (np.array(y_true) == np.array(y_pred)).mean()
print(f"\nTest Accuracy: {accuracy * 100:.2f}%")



Test Accuracy: 95.03%


In [17]:
# ---- Save ----
torch.save(model.state_dict(), "hybrid_baseline_cifake.pth")

In [18]:
# Adversarial Attack Functions
def fgsm_attack(model, images, labels, epsilon=0.05):
    images.requires_grad = True
    outputs = model(images)
    loss = F.cross_entropy(outputs, labels)
    model.zero_grad()
    loss.backward()
    
    # Create adversarial examples
    attack_images = images + epsilon * images.grad.sign()
    attack_images = torch.clamp(attack_images, 0, 1)
    return attack_images.detach()

def pgd_attack(model, images, labels, epsilon=0.03, alpha=0.01, iters=10):
    original_images = images.clone().detach()
    
    for _ in range(iters):
        images.requires_grad = True
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        model.zero_grad()
        loss.backward()
        
        adv_images = images + alpha * images.grad.sign()
        eta = torch.clamp(adv_images - original_images, min=-epsilon, max=epsilon)
        images = torch.clamp(original_images + eta, 0, 1).detach()
    
    return images


In [23]:

# Adversarial Evaluation Function
def evaluate_attacks(model, loader, attack_fn, **attack_kwargs):
    model.eval()
    correct = 0
    total = 0
    
    for images, labels in tqdm(loader, desc="Evaluating Attacks"):
        images, labels = images.to(device), labels.to(device)
        
        # Generate adversarial examples
        adv_images = attack_fn(model, images, labels, **attack_kwargs)
        
        # Evaluate on adversarial examples
        outputs = model(adv_images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    return accuracy

# Load your trained model
model = HybridModel().to(device)
model.load_state_dict(torch.load("hybrid_baseline_cifake.pth", map_location=device))
model.eval()


/tmp/ipykernel_31/2690867178.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("hybrid_baseline_cifake.pth", map_location=device))


HybridModel(
  (cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (patch_embed): Conv2d(256, 128, kernel_size=(4, 4), stride=(4, 4))
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=128, bias=True)
 

In [20]:

# Evaluate against FGSM attacks
print("\nEvaluating FGSM Attacks:")
for epsilon in [0.01, 0.03, 0.05]:
    fgsm_acc = evaluate_attacks(model, test_loader, fgsm_attack, epsilon=epsilon)
    print(f"FGSM (ε={epsilon}): Accuracy = {fgsm_acc:.2f}%")

# Evaluate against PGD attacks



Evaluating FGSM Attacks:


Evaluating Attacks: 100%|██████████| 313/313 [01:13<00:00,  4.25it/s]


FGSM (ε=0.01): Accuracy = 26.54%


Evaluating Attacks: 100%|██████████| 313/313 [01:13<00:00,  4.27it/s]


FGSM (ε=0.03): Accuracy = 10.21%


Evaluating Attacks: 100%|██████████| 313/313 [01:14<00:00,  4.21it/s]


FGSM (ε=0.05): Accuracy = 20.05%

Evaluating PGD Attacks:


Evaluating Attacks: 100%|██████████| 313/313 [06:41<00:00,  1.28s/it]

PGD : Accuracy = 0.03%
